In [5]:
# general settings
basePath = './'
filename = 'umap_on_brain_activity_distance'
figdir = '%sfig/' %(basePath)
savdir = '%sres/%s/' %(basePath,filename)


#!pip install umap-lear
import os
import numpy as np
import pandas as pd
import umap
import h5py 
from datetime import datetime
import scipy.io as sio
from scipy.spatial import distance
import matplotlib.pyplot as plt
%matplotlib inline

if os.path.exists(savdir) == 0:
    os.makedirs(savdir)


umap.UMAP()


UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=1.0, local_connectivity=1.0, metric='euclidean',
   metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
   n_neighbors=15, negative_sample_rate=5, random_state=None,
   repulsion_strength=1.0, set_op_mix_ratio=1.0, spread=1.0,
   target_metric='categorical', target_metric_kwds=None,
   target_n_neighbors=-1, target_weight=0.5, transform_queue_size=4.0,
   transform_seed=42, verbose=False)

In [ ]:
# start analysis
sbjIDs = ['Subject1','Subject2','Subject3','Subject4','Subject5','Average']

# UMAP parameters
n_neighbour = 20
min_dist = 0.5
seed = 1
spread = 10
nclust = 27

for sbjID in sbjIDs:
    if sbjID == 'Average':
        sbjIDs_temp = ['Subject1','Subject2','Subject3','Subject4','Subject5']
        for sbjID2 in sbjIDs_temp:
            fname = '%sres/emotion2020_analysis_BATCH/kmeans_clustering/%s/nCV6_WholeBrainEnc/kmeans_%dclust_top5_res.mat' %(basePath,sbjID2,nclust)
            d =  h5py.File(fname)
            if sbjID2 == 'Subject1':
                dist = d['corrdist_vox'].value
                for i in range(dist.shape[0]):
                    dist[i,i] = 0.0 
            else:
                dist_sub = d['corrdist_vox'].value
                for i in range(dist_sub.shape[0]):
                    dist_sub[i,i] = 0.0 
                dist = dist + dist_sub
        dist = dist/len(sbjIDs_temp)
        
    else:
        fname = '%sres/emotion2020_analysis_BATCH/kmeans_clustering/%s/nCV6_WholeBrainEnc/kmeans_%dclust_top5_res.mat' %(basePath,sbjID,nclust)
        d =  h5py.File(fname)
        dist = d['corrdist_vox'].value
        for i in range(dist.shape[0]):
            dist[i,i] = 0.0 

    if os.path.exists(savdir) == 0:
        print "Brain distance matrix for %s was not yet prepared." %(sbjID)
    else:

        saveFname    = '%snn%d_md%.1f_sp%d_seed%d_%s.mat' %(savdir,n_neighbour,min_dist,spread,seed,sbjID)
        saveFnameChk = '%snn%d_md%.1f_sp%d_seed%d_%s_log.log' %(savdir,n_neighbour,min_dist,spread,seed,sbjID)
        
        if os.path.exists(saveFnameChk) == 1:
            print('Skip:%s' %(saveFnameChk))
        else:
            print('Start:%s' %(saveFnameChk))
            d = datetime.today()
            print('Start: %s:' %(d))
            ff = open(saveFnameChk,'w')
            ff.write(' ')
            try:
                ff.close()
            except:
                print 'Failed. close ' + saveFnameChk


            try:

                dist_umap_ = umap.UMAP(n_neighbors=n_neighbour,
                                      n_components=2, metric='precomputed', 
                                      random_state=seed, 
                                      min_dist = min_dist,
                                      spread = spread
                                     ).fit(dist)
                dist_umap = dist_umap_.embedding_
                print saveFname
                sio.savemat(saveFname,{'dist_umap':dist_umap})


                plt.figure(figsize=(5,5))
                plt.scatter(dist_umap[:, 0], dist_umap[:, 1], s= 5,c='black')
                plt.axis('image')
                plt.show()

                print saveFname
                sio.savemat(saveFname,{'dist_umap':dist_umap})
            except:
                print 'error'





